# HierarchicalForecast with GluonTS Example Notebook


This is an example notebook which shows how HierarchicalForecast's reconciliation capabilities can be integrated with other popular machine learning libraries, in this case GluonTS. 

It trains the GluonTS DeepAREstimator on the TourismLarge Hierarchical Dataset, then uses the `samples_to_quantiles_df` util function to transform the output forecasts into a dataframe compatible with HierarchicalForecast's reconciliation functions.

## 1. Installing packages

In [1]:
%%capture
!pip install gluonts
!pip install pytorch_lightning
!pip install datasetsforecast
!pip install git+https://github.com/Nixtla/hierarchicalforecast.git

In [2]:
!pip install mxnet-cu112
import mxnet as mx
mx.context.num_gpus()

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


1

In [3]:
from datasetsforecast.hierarchical import HierarchicalData
from gluonts.dataset.pandas import PandasDataset
from gluonts.mx.model.deepar import DeepAREstimator
from gluonts.mx.trainer import Trainer
from gluonts.evaluation import make_evaluation_predictions

from hierarchicalforecast.methods import BottomUp, MinTrace
from hierarchicalforecast.core import HierarchicalReconciliation
from hierarchicalforecast.evaluation import scaled_crps
from hierarchicalforecast.utils import samples_to_quantiles_df

import pandas as pd
import numpy as np

/usr/local/lib/python3.10/dist-packages/gluonts/json.py:101: UserWarning: Using `json`-module for json-handling. Consider installing one of `orjson`, `ujson` to speed up serialization and deserialization.
  warnings.warn(


## 2. Load hierarchical dataset


This detailed Australian Tourism Dataset comes from the National Visitor Survey, managed by the Tourism Research Australia, it is composed of 555 monthly series from 1998 to 2016, it is organized geographically, and purpose of travel. The natural geographical hierarchy comprises seven states, divided further in 27 zones and 76 regions. The purpose of travel categories are holiday, visiting friends and relatives (VFR), business and other. The MinT (Wickramasuriya et al., 2019), among other hierarchical forecasting studies has used the dataset it in the past. The dataset can be accessed in the [MinT reconciliation webpage](https://robjhyndman.com/publications/mint/), although other sources are available.

| Geographical Division | Number of series per division | Number of series per purpose | Total |
|          ---          |               ---             |              ---             |  ---  |
|  Australia            |              1                |               4              |   5   |
|  States               |              7                |              28              |  35   |
|  Zones                |             27                |              108             |  135  |
|  Regions              |             76                |              304             |  380  |
|  Total                |            111                |              444             |  555  |


In [4]:
dataset = 'TourismLarge'
Y_df, S_df, tags = HierarchicalData.load(directory = "./data", group=dataset)
Y_df['ds'] = pd.to_datetime(Y_df['ds'])

In [5]:
def sort_hier_df(Y_df, S_df):
    # sorts unique_id lexicographically
    Y_df.unique_id = Y_df.unique_id.astype('category')
    Y_df.unique_id = Y_df.unique_id.cat.set_categories(S_df.index)
    Y_df = Y_df.sort_values(by=['unique_id', 'ds'])
    return Y_df

Y_df = sort_hier_df(Y_df, S_df)

In [6]:
horizon = 12

Y_test_df = Y_df.groupby('unique_id').tail(horizon)
Y_train_df = Y_df.drop(Y_test_df.index)
Y_train_df

,unique_id,ds,y
0,TotalAll,1998-01-01,45151.071280
1,TotalAll,1998-02-01,17294.699551
2,TotalAll,1998-03-01,20725.114184
3,TotalAll,1998-04-01,25388.612353
4,TotalAll,1998-05-01,20330.035211
...,...,...,...
126523,GBDOth,2015-08-01,17.683774
126524,GBDOth,2015-09-01,0.000000
126525,GBDOth,2015-10-01,0.000000
126526,GBDOth,2015-11-01,0.000000


In [7]:
ds = PandasDataset.from_long_dataframe(Y_train_df, target="y", item_id="unique_id")

## 3. Fit and Predict Model


In [8]:
estimator = DeepAREstimator(
    freq="M",
    prediction_length=horizon,
    trainer=Trainer(ctx = mx.context.gpu(),
                    epochs=20),
)
predictor = estimator.train(ds)

forecast_it = predictor.predict(ds, num_samples=1000)

forecasts = list(forecast_it)
forecasts = np.array([arr.samples for arr in forecasts])
forecasts.shape

100%|██████████| 50/50 [00:03<00:00, 13.29it/s, epoch=20/20, avg_epoch_loss=5]


(555, 1000, 12)

## 4. Reconciliation


In [9]:
level = np.arange(1, 100, 2)

#transform the output of DeepAREstimator to a form that is compatible with HierarchicalForecast
quantiles, forecast_df = samples_to_quantiles_df(samples=forecasts, 
                               unique_ids=S_df.index, 
                               dates=Y_test_df['ds'].unique(), 
                               level=level,
                               model_name='DeepAREstimator')

#reconcile forecasts
reconcilers = [
    BottomUp(),
    MinTrace('ols')
]
hrec = HierarchicalReconciliation(reconcilers=reconcilers)

forecast_rec = hrec.reconcile(Y_hat_df=forecast_df, S=S_df, tags=tags, level=level)

In [10]:
forecast_rec

,ds,DeepAREstimator,DeepAREstimator-median,DeepAREstimator-lo-99,DeepAREstimator-lo-97,DeepAREstimator-lo-95,DeepAREstimator-lo-93,DeepAREstimator-lo-91,DeepAREstimator-lo-89,DeepAREstimator-lo-87,...,DeepAREstimator/MinTrace_method-ols-hi-81,DeepAREstimator/MinTrace_method-ols-hi-83,DeepAREstimator/MinTrace_method-ols-hi-85,DeepAREstimator/MinTrace_method-ols-hi-87,DeepAREstimator/MinTrace_method-ols-hi-89,DeepAREstimator/MinTrace_method-ols-hi-91,DeepAREstimator/MinTrace_method-ols-hi-93,DeepAREstimator/MinTrace_method-ols-hi-95,DeepAREstimator/MinTrace_method-ols-hi-97,DeepAREstimator/MinTrace_method-ols-hi-99
unique_id,,,,,,,,,,,,,,,,,,,,,
TotalAll,2016-01-01,43165.929688,43002.058594,27712.297979,30371.243516,32741.458740,33305.429492,34446.465957,35164.380410,35732.592422,...,48703.132046,48956.752480,49233.843836,49540.743219,49886.826218,50286.877928,50766.394577,51375.717577,52240.506366,53910.351214
TotalAll,2016-02-01,20326.796875,20469.210938,13156.550879,15086.488257,15738.457031,16134.386343,16696.160010,16828.676436,17139.442129,...,22902.635244,23019.412684,23146.997118,23288.306411,23447.657478,23631.857993,23852.647485,24133.205242,24531.390118,25300.256426
TotalAll,2016-03-01,24362.203125,24237.250977,17340.837197,18470.071582,19132.180615,19658.168945,19974.223359,20339.483584,20519.382959,...,26759.166634,26873.896338,26999.243530,27138.074912,27294.631699,27475.602189,27692.520055,27968.158127,28359.360682,29114.744632
TotalAll,2016-04-01,29131.662109,29236.008789,19923.623740,21814.112246,22685.987500,23350.113418,23721.056963,24168.286201,24513.198066,...,32277.209370,32427.584386,32591.875632,32773.840464,32979.037796,33216.233913,33500.545877,33861.821798,34374.566871,35364.640665
TotalAll,2016-05-01,22587.779297,22638.541016,14453.285947,16236.985869,17163.251807,17894.046758,18559.204453,18789.053066,19055.381455,...,25400.976716,25532.984575,25677.208902,25836.948122,26017.082170,26225.306596,26474.892029,26792.040860,27242.158045,28111.301901
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
GBDOth,2016-08-01,-0.300811,-0.316894,-2.994549,-2.208182,-2.005075,-1.725068,-1.620723,-1.501304,-1.355108,...,27.151595,28.293141,29.540330,30.921685,32.479405,34.280039,36.438344,39.180908,43.073324,50.589300
GBDOth,2016-09-01,-0.089410,-0.079164,-2.981229,-2.356738,-1.812428,-1.499515,-1.365453,-1.199702,-1.120727,...,24.912080,26.035044,27.261932,28.620801,30.153165,31.924489,34.047662,36.745584,40.574640,47.968273
GBDOth,2016-10-01,-0.196041,-0.207104,-2.829650,-2.270969,-1.674091,-1.289834,-1.153728,-1.078916,-1.029915,...,25.423958,26.550973,27.782287,29.146059,30.683952,32.461666,34.592499,37.300154,41.143025,48.563331


## 5. Evaluation

In [11]:
rec_model_names = ['DeepAREstimator/MinTrace_method-ols', 'DeepAREstimator/BottomUp']

quantiles = np.array(quantiles[1:]) #remove first quantile (median)
n_quantiles = len(quantiles)
n_series = len(S_df)

for name in rec_model_names:
    quantile_columns = [col for col in forecast_rec.columns if (name+'-') in col]
    y_rec  = forecast_rec[quantile_columns].values 
    y_test = Y_test_df['y'].values

    y_rec  = y_rec.reshape(n_series, horizon, n_quantiles)
    y_test = y_test.reshape(n_series, horizon)
    scrps  = scaled_crps(y=y_test, y_hat=y_rec, quantiles=quantiles)
    print("{:<40} {:.5f}".format(name+":", scrps))

DeepAREstimator/MinTrace_method-ols:     0.12632
DeepAREstimator/BottomUp:                0.13933
